In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = 'supplied-data/police_press_releases.csv'
df = pd.read_csv(file_path, parse_dates = ['date_published', 'date_modified'])
df['date_published'] = df['date_published'].dt.date
df['date_modified'] = df['date_modified'].dt.date
pd.set_option('display.max_colwidth', None)  # show entire content column
pd.set_option('display.width', None)         # avoid line breaks
pd.set_option('display.max_columns', None)   # show all columns
display(df)

,title,date_published,date_modified,content
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing."
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing."
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing."
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongoing."
4,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the evening (2045hrs), the Police were informed of a traffic accident in Triq il-Buqana, Rabat.Preliminary investigations showed that a collision occurred between a Kia Avella driven by a 25-year-old man from Santa Luċija and a Kymco Agility motorcycle driven by a 41-year-old man from Birkirkara.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing."
...,...,...,...,...
106,Motorcycle accident in Attard,2025-02-05,2025-02-05,"A 52-year-old man and residing in Ħaż-Żebbuġ, at around 0900hrs, was taken to Mater Dei Hospital because he was injured in a traffic accident in Vjal L-Istadium Nazzjonali in Ta’ Qali, Attard.Preliminary investigations showed that the man was riding a Kymco motorcycle, lost control and fell.He was assisted by a medical team, taken to hospital by ambulance and certified to have sustained grievous injuries.Police investigations are ongoing."
107,Naxxar traffic accident,2024-12-19,2024-12-19,"Today, at around 1045hrs, the Police were informed of a traffic accident in Triq il-Ġermanja, Naxxar.The Police went immediately on site and preliminary investigations showed that a 78-year-old woman who resides in Naxxar, lost control of the Toyota Vitz she was driving and crashed into a parked truck.The vi

In [3]:
import re

UPDATED_HDR = re.compile(r"Updated\s+[A-Za-z]*\s*\d{1,2}[:.]?\d{0,2}\s?(?:am|pm)|Updated\s+\d{1,2}[:.]?\d{0,2}\s?(?:am|pm)", re.IGNORECASE)

TRAFFIC_WORDS = re.compile(
    r"\b(traffic accident|road accident|collision|crash|hit[- ]?by|run[- ]?over|lost control|skidded)\b",
    re.IGNORECASE
)

VEHICLE_WORDS = re.compile(
    r"\b(car|vehicle|van|bus|truck|lorry|motorcycle|motorbike|motor[- ]?cycle|bike|scooter|bicycle|cyclist|pedestrian)\b",
    re.IGNORECASE
)
CAR_BRANDS = re.compile(
    r"\b(peugeot|volkswagen|vw|mercedes|bmw|audi|toyota|kia|ford|hyundai|nissan|mazda|honda)\b",
    re.IGNORECASE
)

def strip_updated(text: str) -> str: #removing Updated from the text
    if not isinstance(text, str):
        return ''
    return UPDATED_HDR.sub('', text)

def refers_to_traffic_accident(text: str) -> int:
    if not isinstance(text, str):
        return 0
    t = strip_updated(text)
    has_traffic_event = bool(TRAFFIC_WORDS.search(t))
    has_vehicle = bool(VEHICLE_WORDS.search(t) or CAR_BRANDS.search(t))
    return 1 if has_traffic_event and has_vehicle else 0

df['traffic_accident_flag'] = df['content'].apply(refers_to_traffic_accident)
df['traffic_accident_flag'].value_counts()

traffic_accident_flag
1    103
0      8
Name: count, dtype: int64

In [4]:
from datetime import datetime, timedelta
from dateutil import parser
import re
def extract_date_description(content):
    content_lower = content.lower()

    # Relative dates
    if "yesterday" in content_lower:
        return "yesterday"
    if "today" in content_lower:
        return "today"
    if "tomorrow" in content_lower:
        return "tomorrow"

    # Written date pattern example: "Wednesday 11th June"
    written_date_pattern = r"(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\s+\d{1,2}(st|nd|rd|th)?\s+[a-zA-Z]+"
    match = re.search(written_date_pattern, content_lower)

    if match:
        return match.group(0)

    return None


# --------------------------------------------------------
# Handle relative dates
# --------------------------------------------------------
def parse_relative_date(description, published_date):

    # Convert published_date (a python date) → datetime
    published_dt = datetime.combine(published_date, datetime.min.time())

    if description == "yesterday":
        return published_dt - timedelta(days=1)

    if description == "today":
        return published_dt  # Use the press release date, not system date

    if description == "tomorrow":
        return published_dt + timedelta(days=1)

    return None


# --------------------------------------------------------
# Handle written dates like "Wednesday 11th June"
# --------------------------------------------------------
def parse_written_date(description):
    if description is None:
        return None
    try:
        return parser.parse(description, fuzzy=True, dayfirst=True)
    except:
        return None


# --------------------------------------------------------
# Fallback: convert published date (already a date)
# --------------------------------------------------------
def fallback_published_date(published_date):
    return datetime.combine(published_date, datetime.min.time())


# --------------------------------------------------------
# Main normalization function
# --------------------------------------------------------
def normalize_date(row):
    content = row["content"]
    published_date = row["date_published"]   # already a python date

    # 1️⃣ Extract description from content
    description = extract_date_description(content)

    # 2️⃣ Try relative dates first
    rel = parse_relative_date(description, published_date)
    if rel:
        return rel

    # 3️⃣ Try written date
    written = parse_written_date(description)
    if written:
        return written

    # 4️⃣ Fallback: use the press release date
    return fallback_published_date(published_date)



# --------------------------------------------------------
# Apply to your existing dataframe
# --------------------------------------------------------
df["normalized_date"] = df.apply(normalize_date, axis=1)
df["day_of_week"] = df["normalized_date"].dt.strftime("%A")
print (df["day_of_week"])

0       Thursday
1       Thursday
2         Monday
3        Tuesday
4         Sunday
         ...    
106    Wednesday
107     Thursday
108       Sunday
109     Thursday
110      Tuesday
Name: day_of_week, Length: 111, dtype: object


In [5]:
same_day_pattern = re.compile(r"\b(today|same day)\b", re.IGNORECASE)
previous_day_pattern = re.compile(r"\b(yesterday|previous day|last night|before midnight)\b", re.IGNORECASE)
weekday_pattern = re.compile(r"\b(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\b", re.IGNORECASE)

accident_day_list = []
for idx, row in df.iterrows():
    content = str(row['content'])
    publish_date = pd.to_datetime(row['date_published'], errors='coerce', dayfirst=True)
    classification = "undetermined"

    if same_day_pattern.search(content):
        classification = "same day"
    elif previous_day_pattern.search(content):
        classification = "previous day"
    else:
        weekday_match = weekday_pattern.search(content)
        if weekday_match and publish_date is not pd.NaT:
            mentioned_day = weekday_match.group(0).lower()
            weekdays = {'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 'friday': 4, 'saturday': 5, 'sunday': 6}
            mentioned_num = weekdays.get(mentioned_day)
            publish_num = publish_date.weekday()
            if mentioned_num == publish_num:
                classification = "same day"
            elif (publish_num - mentioned_num) % 7 == 1:
                classification = "previous day"
    accident_day_list.append(classification)
df['accident_day'] = accident_day_list

In [6]:
def extract_time(text):
    match = re.search(r'\b(\d{1,2})(?::?)(\d{2})\s*hrs\b', text, flags=re.IGNORECASE)
    if match:
        hour = int(match.group(1))
        minute = int(match.group(2))
        # Validate 24-hour time
        if 0 <= hour <= 23 and 0 <= minute <= 59:
            return f"{hour:02d}:{minute:02d}"
    return None
df['time_24h'] = df['content'].apply(extract_time)
print(df['time_24h'])

0      09:30
1      18:30
2      08:00
3      18:00
4      20:45
       ...  
106    09:00
107    10:45
108    08:00
109    22:15
110    07:00
Name: time_24h, Length: 111, dtype: object


In [7]:
def extract_location(text):
    match = re.search(r'\b(?:on|in|at)\s+([A-Z][\w\s\'’\,\-]*)(?=[\.,])', text)
    if match:
        return match.group(1).strip()
    return None
df['location'] = df['content'].apply(extract_location)
print(df['location'])

0                      Triq il-Belt Valletta, Żurrieq
1                         Triq Dawret il-Gudja, Gudja
2                                  Valley Road, Qormi
3                      Triq Dawret Ħal Għaxaq, Għaxaq
4                               Triq il-Buqana, Rabat
                            ...                      
106    Vjal L-Istadium Nazzjonali in Ta’ Qali, Attard
107                          Triq il-Ġermanja, Naxxar
108                           Vjal il-Helsien, Zebbug
109           Triq il-Wied ta’ Birkirkara, Birkirkara
110                     St Luke’s Square, Gwardamanġa
Name: location, Length: 111, dtype: object


In [8]:
car_brands = ['Toyota', 'Honda', 'BMW', 'Ford', 'Mercedes', 'Audi', 
              'Nissan', 'Fiat', 'Volkswagen', 'Volvo', 'Kymco', 
              'Mazda', 'Peugeot']

def extract_car_brand(text): 
    for brand in car_brands: 
        if brand.lower() in text.lower(): 
            return brand 
    return None

df['car_brand'] = df['content'].apply(extract_car_brand)
print(df['car_brand'])

0        Toyota
1         Honda
2         Honda
3         Volvo
4         Kymco
         ...   
106       Kymco
107      Toyota
108     Peugeot
109       Mazda
110    Mercedes
Name: car_brand, Length: 111, dtype: object


In [9]:
def extract_traffic (text):
    traffic_keywords = [
        'heavy traffic',
        'traffic built up',
        'traffic congestion',
        'traffic was disrupted',
        'traffic flow',
        'lanes were closed',
        'road was closed',
        'road closure',
        'vehicles were stuck',
        'standstill traffic',
        'traffic delays',
        'slow traffic',
        'major traffic',
        'route was blocked',
        'road was blocked',
        'closed to traffic'
    ]
    for keyword in traffic_keywords:
        if keyword in text.lower():
            return 'Traffic'
    return 'No Traffic'
df['traffic_status'] = df['content'].apply(extract_traffic)
print(df['traffic_status'])


0      No Traffic
1      No Traffic
2      No Traffic
3      No Traffic
4      No Traffic
          ...    
106    No Traffic
107    No Traffic
108    No Traffic
109    No Traffic
110    No Traffic
Name: traffic_status, Length: 111, dtype: object


In [10]:
display(df)

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-07-29,Tuesday,previous day,18:00,"Triq Dawret Ħal Għaxaq, Għaxaq",Volvo,No Traffic
4,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the evening (2045hrs), the Police were informed of a traffic accident in Triq il-Buqana, Rabat.Preliminary investigations showed that a collision occurred between a Kia Avella driven by a 25-year-old man from Santa Luċija and a Kymco Agility motorcycle driven by a 41-year-old man from Birkirkara.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-04-06,Sunday,previous day,20:45,"Triq il-Buqana, Rabat",Kymco,No Traffic
...,...,...,...,...,...,...,...,...,...,...,...,...
106,Motorcycle accident in Attard,2025-02-05,2025-02-05,"A 52-year-old man and residing in Ħaż-Żebbuġ, at around 0900hrs, was taken to Mater Dei Hospital because he was injured in a traffic accident in Vjal L-Istadium Nazzjonali in Ta’ Qali, Attard.Preliminary investigations showed that the man was ridi

In [11]:
patterns = {
    'no injuries': re.compile(r"\b(no injuries|uninjured|escaped injury)\b", re.IGNORECASE),
    'minor': re.compile(r"\b(minor|minor injuries|slight injuries|light injuries)\b", re.IGNORECASE),
    'serious': re.compile(r"\b(serious|seriously injured)\b", re.IGNORECASE),
    'grievous': re.compile(r"\b(grievous|grievously injured)\b", re.IGNORECASE),
    'critical': re.compile(r"\b(critical|critically injured|critical condition)\b", re.IGNORECASE),
    'death': re.compile(r"\b(died|death|dead|fatal|succumbed|killed)\b", re.IGNORECASE)
}

severity_list = []
for content in df['content'].astype(str):
    severity = 'unknown'
    for category, pattern in patterns.items():
        if pattern.search(content):
            severity = category
            break
    severity_list.append(severity)

df['injury_severity'] = severity_list
display(df)

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-07-29,Tuesday,previous day,18:00,"Triq Dawret Ħal Għaxaq, Għaxaq",Volvo,No Traffic,grievous
4,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the evening (2045hrs), the Police were informed of a traffic accident in Triq il-Buqana, Rabat.Preliminary investigations showed that a collision occurred between a Kia Avella driven by a 25-year-old man from Santa Luċija and a Kymco Agility motorcycle driven by a 41-year-old man from Birkirkara.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-04-06,Sunday,previous day,20:45,"Triq il-Buqana, Rabat",Kymco,No Traffic,grievous
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Motorcycle accident in Attard,2025-02-05,2025-02-05,"A 52-year-old man and residing in Ħaż-Żebbuġ, at around 0900hrs, was taken to Mater Dei Hospital because he was injured in a traffic accident in Vjal L-Istadium Nazzjonali in Ta’ Qali

In [12]:
# terms
PED_TERMS = r"\b(pedestrian|walker|passer[- ]?by|passerby|foot\s*traveller|on foot)\b"
CYCLIST_TERMS = r"\b(cyclist|bicyclist|bicycle|push\s?bike|pedal\s?cycle|e\-?bike|ebike)\b"
MOTORCYCLE_TERMS = r"\b(motorcyclist|motorcycle|motor\s?bike|motorbike|bike\b(?!\s?ride)|biker)\b"

# phrases indicating being hurt
HURT_TERMS = r"\b(hurt|injur(?:ed|ies)|grievously|critically|serious(?:ly)?|run over|hit|struck|knocked down|hospitalis(?:ed|ed))\b"
# traffic context keywords
TRAFFIC_TERMS = r"\b(road|street|triq|lane|bypass|tunnel|roundabout|junction|seafront|coast road|regional road|traffic|collision|accident|crash)\b"

PED_RX = re.compile(PED_TERMS, re.IGNORECASE)
CYCLIST_RX = re.compile(CYCLIST_TERMS, re.IGNORECASE)
MOTO_RX = re.compile(MOTORCYCLE_TERMS, re.IGNORECASE)

HURT_RX = re.compile(HURT_TERMS, re.IGNORECASE)
TRAFFIC_RX = re.compile(TRAFFIC_TERMS, re.IGNORECASE)

def mark_motorcyclist(text: str) -> str:
    t = strip_updated(text)
    if MOTO_RX.search(t) and HURT_RX.search(t) and TRAFFIC_RX.search(t) and not CYCLIST_RX.search(t):
        return 'motorcyclist'
    return ''

def mark_cyclist(text: str) -> str:
    t = strip_updated(text)
    # Must mention a cyclist and harm in traffic context, but NOT a motorcyclist
    if CYCLIST_RX.search(t) and HURT_RX.search(t) and TRAFFIC_RX.search(t) and not MOTO_RX.search(t):
        return 'cyclist'
    # Catch constructions like "was cycling / on a bicycle ... and was hit"
    cycling_phrase = re.search(
        r"\b(cycling|on\s+a\s+bicycle|on\s+his\s+bicycle|on\s+her\s+bicycle|on\s+a\s+push\s?bike)\b",
        t, re.IGNORECASE
    )
    if cycling_phrase and HURT_RX.search(t) and TRAFFIC_RX.search(t) and not MOTO_RX.search(t):
        return 'cyclist'
    return ''

def mark_pedestrian(text: str) -> str:
    t = strip_updated(text)
    if PED_RX.search(t) and HURT_RX.search(t) and TRAFFIC_RX.search(t):
        return 'pedestrian'
    # also catch constructions like "a woman was run over" without the word pedestrian
    if re.search(r"\b(run over|knocked down|hit|struck)\b", t, re.IGNORECASE) and \
       re.search(r"\b(man|woman|boy|girl|elderly|child|teenager|youth|person)\b", t, re.IGNORECASE) and \
       TRAFFIC_RX.search(t):
        # ensure it's not inside vehicle context (driver etc.) by simple heuristic
        if not re.search(r"\b(driver|motorist|cyclist|motorcyclist)\b", t, re.IGNORECASE):
            return 'pedestrian'
    return ''

# Apply
df['pedestrian'] = df['content'].apply(mark_pedestrian)
df['motorcyclist'] = df['content'].apply(mark_motorcyclist)
df['cyclist'] = df['content'].apply(mark_cyclist)
df

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,pedestrian,motorcyclist,cyclist
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,,motorcyclist,
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,,motorcyclist,
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous,,motorcyclist,
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-07-29,Tuesday,previous day,18:00,"Triq Dawret Ħal Għaxaq, Għaxaq",Volvo,No Traffic,grievous,,motorcyclist,
4,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the evening (2045hrs), the Police were informed of a traffic accident in Triq il-Buqana, Rabat.Preliminary investigations showed that a collision occurred between a Kia Avella driven by a 25-year-old man from Santa Luċija and a Kymco Agility motorcycle driven by a 41-year-old man from Birkirkara.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-04-06,Sunday,previous day,20:45,"Triq il-Buqana, Rabat",Kymco,No Traffic,grievous,,motorcyclist,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Motorcycle accident in Attard,2025-02-05,2025-02-05,"A 52-year-old man and residing in Ħaż-Żebbuġ, at around 0900hrs,

In [13]:
idx_pedestrian = df.loc[(df['pedestrian'] == 'pedestrian') & (df['motorcyclist'] != 'motorcyclist') & (df['cyclist'] != 'cyclist')].index
#idx_pedestrian
df.loc[idx_pedestrian, 'affected_party'] = 'pedestrian'

idx_motorcyclist = df.loc[(df['motorcyclist'] == 'motorcyclist')].index
df.loc[idx_motorcyclist, 'affected_party'] = 'motorcyclist'

idx_cyclist = df.loc[(df['cyclist'] == 'cyclist') & (df['motorcyclist'] != 'motorcyclist')].index
df.loc[idx_cyclist, 'affected_party'] = 'cyclist'

idx_driver = df.loc[(df['pedestrian'] != 'pedestrian') & (df['motorcyclist'] != 'motorcyclist') & (df['cyclist'] != 'cyclist') & (df['traffic_accident_flag'] == 1)].index
df.loc[idx_driver, 'affected_party'] = 'driver'

df.drop(['pedestrian', 'motorcyclist', 'cyclist'], axis=1, inplace=True)

df

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,affected_party
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,motorcyclist
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,motorcyclist
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous,motorcyclist
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-07-29,Tuesday,previous day,18:00,"Triq Dawret Ħal Għaxaq, Għaxaq",Volvo,No Traffic,grievous,motorcyclist
4,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the evening (2045hrs), the Police were informed of a traffic accident in Triq il-Buqana, Rabat.Preliminary investigations showed that a collision occurred between a Kia Avella driven by a 25-year-old man from Santa Luċija and a Kymco Agility motorcycle driven by a 41-year-old man from Birkirkara.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-04-06,Sunday,previous day,20:45,"Triq il-Buqana, Rabat",Kymco,No Traffic,grievous,motorcyclist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Motorcycle accident in Attard,2025-02-05,2025-02-05,"A 52-year-old man and residing in Ħaż-Żebbuġ, at around 0900hrs, was taken to Mater Dei Hospital be

In [14]:
from typing import List

# ---------------------------------------------------------------------
# 1) Lexicons and regex patterns
## ---------------------------------------------------------------------

# Vehicles: car types, brands, and common vehicle nouns
VEHICLES = r"""
car|van|truck|pickup|vehicle|bus|jeep|coach|minivan|school\s*van|
mercedes|bmw|toyota|nissan|peugeot|skoda|mazda|ford|renault|citroen|
suzuki|isuzu|honda|kymco|aprilia|yamaha|benelli|dacia|smart|passo|
sprinter|master|fiesta|civic|swift|fit|starlet|funcargo|aygo|hilux|
volvo|jaguar|optare|otokar|dac|vitz|demio|cx-?3|c180|xf|b\s*max|
gpd125-?a|tweet|vespa|piaggio
"""

# Fixed obstacles (roadside infrastructure and static objects)
OBSTACLES = r"""
wall|tree|tunnel|light\s*post|lamp\s*post|barrier|crash\s*barrier|bollard|
building|house|signpost|pillar|electricity\s*pole|skip|stationary\s*skip|
rubble\s*wall|bridge|guardrail|canopy|
"""

# Person terms likely to indicate pedestrians (on foot)
PERSON_WORDS = r"""
pedestrian|man|woman|boy|girl|child|toddler|elderly|teen(?:ager)?|
people|passer\s*by|passers\s*by|\d{1,3}\s*-?\s*year\s*-?\s*old
"""

# Rider/driver terms (exclude these from pedestrian struck logic)
RIDER_WORDS = r"""
cyclist|bicycl(?:e|ist)|motorcyclist|rider|driver
"""

# Accident/impact verbs and phrases
V_CRASH = r"crash(?:ed|es|ing)?|collid(?:e|ed|ing)|smashed\s+into|ramm(?:ed|ing)|hit|struck|slammed|clip(?:ped|s)"
V_RUN_OVER = r"run(?:\s*-)over|ran\s*over|run\s+over"
V_LOST_CONTROL = r"lost\s+control|went\s+out\s+of\s+control|careen(?:ed|ing)|skidd(?:ed|ing)|went\s+off\s+the\s+road|fell\s+off\s+(?:the\s+)?(?:bike|motorcycle|motorbike)"
V_OVERTURN = r"overturn(?:ed|s|ing)|rolled\s+over|flip(?:ped|s|ping)|ended\s+up\s+on\s+its\s+side"

# Compile regexes
P_VEHICLE = re.compile(VEHICLES, re.I | re.X)
P_OBSTACLE = re.compile(OBSTACLES, re.I | re.X)
P_PERSON = re.compile(PERSON_WORDS, re.I | re.X)
P_RIDER = re.compile(RIDER_WORDS, re.I | re.X)
P_CRASH = re.compile(V_CRASH, re.I)
P_RUN_OVER = re.compile(V_RUN_OVER, re.I)
P_LOST_CONTROL = re.compile(V_LOST_CONTROL, re.I)
P_OVERTURN = re.compile(V_OVERTURN, re.I)
P_GENERIC_ACCIDENT = re.compile(r"\b(accident|incident|injur(?:y|ies))\b", re.I)

# ---------------------------------------------------------------------
# 2) Sentence splitter (simple, rule-based)
# ---------------------------------------------------------------------
def split_sentences(t: str) -> List[str]:
    if not isinstance(t, str):
        return []
    return [s.strip() for s in re.split(r"(?<=[.!?])\s+|\n+", t) if s.strip()]

# ---------------------------------------------------------------------
# 3) Per-sentence classification
#    Returns granular flags for the sentence.
# ---------------------------------------------------------------------
def classify_sentence(s: str):
    s_low = s.lower()
    has_vehicle = bool(P_VEHICLE.search(s_low))
    has_obstacle = bool(P_OBSTACLE.search(s_low))
    has_person = bool(P_PERSON.search(s_low))
    has_rider = bool(P_RIDER.search(s_low))
    has_crash = bool(P_CRASH.search(s_low))
    has_run_over = bool(P_RUN_OVER.search(s_low))
    has_lost_control = bool(P_LOST_CONTROL.search(s_low))
    has_overturn = bool(P_OVERTURN.search(s_low))

    # Pedestrian struck/run-over: person + (hit/struck/run-over) + vehicle, excluding rider contexts
    running_over_pedestrian = 1 if (has_person and (has_run_over or has_crash) and has_vehicle and not has_rider) else 0

    # Collision with fixed object: crash verb + obstacle
    crushed_into_obstacle = 1 if (has_crash and has_obstacle) else 0

    # Overturned: rollover phrases
    overturned = 1 if has_overturn else 0

    # Lost control: phrases directly
    lost_control = 1 if has_lost_control else 0

    # Vehicle–vehicle collision: crash verb + >= 2 mentions of vehicles in the sentence
    vehicle_mentions = len(re.findall(VEHICLES, s_low, re.I | re.X))
    collision_vv = 1 if (has_crash and vehicle_mentions >= 2 and not running_over_pedestrian and not crushed_into_obstacle) else 0

    # Generic collision: crash verb + some vehicle/rider context, excluding pedestrian/obstacle cases
    collision_generic = 1 if (has_crash and (has_vehicle or has_rider) and not (running_over_pedestrian or crushed_into_obstacle)) else 0

    return {
        'collision_vv': collision_vv,
        'collision_generic': collision_generic,
        'running_over_pedestrian': running_over_pedestrian,
        'lost_control': lost_control,
        'crushed_into_obstacle': crushed_into_obstacle,
        'overturned': overturned
    }

# ---------------------------------------------------------------------
# 4) Per-article aggregation + precedence rules
#    Combines sentence flags and applies differentiation rules.
# ---------------------------------------------------------------------
def classify_article(text: str):
    sentences = split_sentences(text)
    flags = {
        'collision': 0,
        'running_over_pedestrian': 0,
        'lost_control': 0,
        'crushed_into_obstacle': 0,
        'overturned': 0,
        'other': 0
    }

    any_accident_mention = False
    saw_vv_collision = False

    for s in sentences:
        if P_GENERIC_ACCIDENT.search(s):
            any_accident_mention = True
        sf = classify_sentence(s)

        # accumulate non-collision flags
        flags['running_over_pedestrian'] |= sf['running_over_pedestrian']
        flags['lost_control'] |= sf['lost_control']
        flags['crushed_into_obstacle'] |= sf['crushed_into_obstacle']
        flags['overturned'] |= sf['overturned']

        # collision handling (prefer vehicle–vehicle over generic)
        if sf['collision_vv']:
            flags['collision'] = 1
            saw_vv_collision = True
        elif sf['collision_generic']:
            flags['collision'] |= 1

    # If obstacle crash detected and collision flagged only by generic sentences, drop collision unless vv also seen
    if flags['crushed_into_obstacle'] and flags['collision'] == 1 and not saw_vv_collision:
        flags['collision'] = 0

    # "Other" only if there is accident context but no specific type matched
    if (flags['collision'] + flags['running_over_pedestrian'] + flags['lost_control'] +
        flags['crushed_into_obstacle'] + flags['overturned'] == 0) and any_accident_mention:
        flags['other'] = 1

    return flags

# ---------------------------------------------------------------------
# 5) Apply improved detection (v2) and save
# ---------------------------------------------------------------------
cols = ['collision','running_over_pedestrian','lost_control','crushed_into_obstacle','overturned','other']
new_flags = df['content'].apply(classify_article)
for c in cols:
    df[c] = new_flags.apply(lambda d: d[c])

#OUT_V2 = 'local_news_articles_with_accident_types.csv'
#df.to_csv(OUT_V2, index=False)

# ---------------------------------------------------------------------
# 6) Create mutually exclusive primary_accident_type (hierarchy)
#    Order can be adjusted to your preference.
# ---------------------------------------------------------------------
# Ensure numeric
for c in cols:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0).astype(int)

# Hierarchy / precedence for single-label assignment
precedence = [
    ('running_over_pedestrian', 'running over pedestrian'),
    ('overturned', 'overturned'),
    ('crushed_into_obstacle', 'crushed into an obstacle'),
    ('collision', 'collision'),
    ('lost_control', 'lost control of the vehicle'),
    ('other', 'other')
]

labels = []
for _, row in df.iterrows():
    label = 'none'
    for col, name in precedence:
        if int(row[col]) == 1:
            label = name
            break
    labels.append(label)

df['primary_accident_type'] = labels

code_map = {
    'running over pedestrian': 'PED',
    'overturned': 'ROLL',
    'crushed into an obstacle': 'FXOBJ',
    'collision': 'COLL',
    'lost control of the vehicle': 'LOC',
    'other': 'OTH',
    'none': 'NONE'
}

df['primary_accident_type_code'] = df['primary_accident_type'].map(code_map)
df


,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,affected_party,collision,running_over_pedestrian,lost_control,crushed_into_obstacle,overturned,other,primary_accident_type,primary_accident_type_code
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,motorcyclist,0,0,0,0,0,1,other,OTH
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-07-29,Tuesday,previous day,18:00,"Triq Dawret Ħal Għaxaq, Għaxaq",Volvo,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH
4,Car-motorcycle collision,2025-04-07,2025-04-07,"Yesterday, at around quarter to nine in the evening (2045hrs), the Police were informed of a traffic accident in Triq il-Buqana, Rabat.Preliminary investigations showed that a collision occurred between a Kia Avella driven by a 25-year-old man from Santa Luċija and a Kymco Agility motorcycle driven by a 41-year-old man from Birkirkara.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-04-06,Sunday,previous day,20:45,"Triq il-Buqana, Rabat",Kymco,No Traffic,grievous,motorc

In [15]:
from datetime import date
def calculate_easter(year):
    a = year % 19
    b = year // 100
    c = year % 100
    d = b // 4
    e = b % 4
    f = (b + 8) // 25
    g = (b - f + 1) // 3
    h = (19 * a + b - d - g + 15) % 30
    i = c // 4
    k = c % 4
    l = (32 + 2*e + 2*i - h - k) % 7
    m = (a + 11*h + 22*l) // 451
    month = (h + l - 7*m + 114) // 31
    day = ((h + l - 7*m + 114) % 31) + 1
    return date(year, month, day)
years = df['normalized_date'].dt.year.unique()
malta_holidays = set()
for year in years:
    # Fixed holidays
    fixed = [
        date(year, 1, 1),
        date(year, 2, 10),
        date(year, 3, 19),
        date(year, 3, 31),
        date(year, 5, 1),
        date(year, 6, 7),
        date(year, 6, 29),
        date(year, 8, 15),
        date(year, 9, 8),
        date(year, 9, 21),
        date(year, 12, 8),
        date(year, 12, 13),
        date(year, 12, 25)
    ]

    # Movable holidays
    easter_sunday = calculate_easter(year)
    good_friday = easter_sunday - timedelta(days=2)

    fixed.append(good_friday)
    fixed.append(easter_sunday)

    malta_holidays.update(fixed)
def is_public_holiday(dt):
    if dt is None:
        return False
    accident_date = dt.date()
    return True if accident_date in malta_holidays else False
df['public_holiday'] = df['normalized_date'].apply(is_public_holiday)
print(df['public_holiday'])

0      False
1      False
2      False
3      False
4      False
       ...  
106    False
107    False
108    False
109    False
110    False
Name: public_holiday, Length: 111, dtype: bool


In [16]:
def extract_accident_type(text):
    text_lower = text.lower()
    if "e-scooter" in text_lower or "escooter" in text_lower:
        return "E-Scooter Accident"

    if "motorcycle" in text_lower or "motorbike" in text_lower or "scooter" in text_lower:
        return "Motorcycle Accident"

    if "pedestrian" in text_lower or "hit by" in text_lower or "run over" in text_lower:
        return "Hit Pedestrian"

    if "collision" in text_lower or "collided" in text_lower or "crash" in text_lower:
        return "Collision"

    if "lost control" in text_lower or "fell" in text_lower:
        return "Lost Control / Fall"

    if "car" in text_lower or "van" in text_lower or "vehicle" in text_lower:
        return "Car Accident"

    return "Other"
df["accident_type"] = df["content"].apply(extract_accident_type)
print (df["accident_type"])

0      Motorcycle Accident
1      Motorcycle Accident
2      Motorcycle Accident
3      Motorcycle Accident
4      Motorcycle Accident
              ...         
106    Motorcycle Accident
107              Collision
108         Hit Pedestrian
109     E-Scooter Accident
110              Collision
Name: accident_type, Length: 111, dtype: object


In [17]:
df[['street', 'locality']] = df.location.str.rsplit(',', n=1, expand=True).apply(lambda x: x.str.strip())
print(df['locality'])
print(df['street'])

0          Żurrieq
1            Gudja
2            Qormi
3           Għaxaq
4            Rabat
          ...     
106         Attard
107         Naxxar
108         Zebbug
109     Birkirkara
110    Gwardamanġa
Name: locality, Length: 111, dtype: object
0                       Triq il-Belt Valletta
1                        Triq Dawret il-Gudja
2                                 Valley Road
3                      Triq Dawret Ħal Għaxaq
4                              Triq il-Buqana
                        ...                  
106    Vjal L-Istadium Nazzjonali in Ta’ Qali
107                          Triq il-Ġermanja
108                           Vjal il-Helsien
109               Triq il-Wied ta’ Birkirkara
110                          St Luke’s Square
Name: street, Length: 111, dtype: object


In [18]:
def parse_time(time_str):
    if not isinstance(time_str, str):
        return None
    try:
        return datetime.strptime(time_str, "%H:%M").time()
    except:
        return None
def get_instance_datetime(row):
    date = row["normalized_date"].date()
    time = parse_time(row["time_24h"])
    if time:
        return datetime.combine(date, time)
    else:
        # If no time present → midnight as fallback
        return datetime.combine(date, datetime.min.time())
df["incident_datetime"] = df.apply(get_instance_datetime, axis=1)
print(df["incident_datetime"])

0     2025-10-09 09:30:00
1     2025-06-19 18:30:00
2     2025-05-12 08:00:00
3     2025-07-29 18:00:00
4     2025-04-06 20:45:00
              ...        
106   2025-02-05 09:00:00
107   2024-12-19 10:45:00
108   2025-03-16 08:00:00
109   2025-07-17 22:15:00
110   2025-08-12 07:00:00
Name: incident_datetime, Length: 111, dtype: datetime64[ns]


In [19]:
import weather

def extract_weather(row):
    dt = row["normalized_date"]
    time_24h = row["time_24h"]
    hour = int(time_24h.split(":")[0]) if time_24h else 0

    rained_today = weather.rain_on_date(dt)
    rained_last_3h = weather.rain_before(dt, hour, 3)

    daily = weather.get_daily_weather_data(dt)
    hourly = weather.get_hourly_weather_data(dt - timedelta(hours=3), dt)

    # Check if data is empty before extracting values
    temp_avg = daily["tavg"].values[0] if not daily.empty and "tavg" in daily else None
    temp_min = daily["tmin"].values[0] if not daily.empty and "tmin" in daily else None
    temp_max = daily["tmax"].values[0] if not daily.empty and "tmax" in daily else None
    precip_mm_day = daily["prcp"].values[0] if not daily.empty and "prcp" in daily else None

    return {
        "rain_today": rained_today,
        "rain_last_3h": rained_last_3h,
        "temp_avg": temp_avg,
        "temp_min": temp_min,
        "temp_max": temp_max,
        "precip_mm_day": precip_mm_day,
        "hourly_records": len(hourly) if hourly is not None else 0
    }
df["weather"] = df.apply(extract_weather, axis=1)
print(df["weather"])

0      {'rain_today': False, 'rain_last_3h': False, 'temp_avg': 20.4, 'temp_min': 16.0, 'temp_max': 25.0, 'precip_mm_day': 0.0, 'hourly_records': 4}
1      {'rain_today': False, 'rain_last_3h': False, 'temp_avg': 24.8, 'temp_min': 21.0, 'temp_max': 29.5, 'precip_mm_day': 0.0, 'hourly_records': 4}
2      {'rain_today': False, 'rain_last_3h': False, 'temp_avg': 21.1, 'temp_min': 17.9, 'temp_max': 25.4, 'precip_mm_day': 0.0, 'hourly_records': 4}
3      {'rain_today': False, 'rain_last_3h': False, 'temp_avg': 25.7, 'temp_min': 22.8, 'temp_max': 29.3, 'precip_mm_day': 0.0, 'hourly_records': 4}
4      {'rain_today': False, 'rain_last_3h': False, 'temp_avg': 15.5, 'temp_min': 11.5, 'temp_max': 19.3, 'precip_mm_day': 0.0, 'hourly_records': 4}
                                                                           ...                                                                      
106      {'rain_today': False, 'rain_last_3h': True, 'temp_avg': 12.1, 'temp_min': 8.7, 'temp_max': 16.1, 

In [29]:
import roadworks

def check_roadworks_near_location(row):
    if pd.isna(row['street']) or pd.isna(row['locality']):
        return False
    return roadworks.had_roadworks(row['locality'], row['street'], row['normalized_date'])

df['roadworks_nearby'] = df.apply(check_roadworks_near_location, axis=1)
df['roadworks_nearby'].describe()

count       100
unique        2
top       False
freq         95
Name: roadworks_nearby, dtype: object

In [21]:
def is_eve_of_public_holiday(dt):
    if dt is None:
        return False
    accident_date = dt.date()
    next_day = accident_date + timedelta(days=1)
    return True if next_day in malta_holidays else False

# Apply to the column
df['eve_of_public_holiday'] = df['normalized_date'].apply(is_eve_of_public_holiday)
print(df['eve_of_public_holiday'])

0      False
1      False
2      False
3      False
4      False
       ...  
106    False
107    False
108    False
109    False
110    False
Name: eve_of_public_holiday, Length: 111, dtype: bool


In [22]:
display(df)

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,affected_party,collision,running_over_pedestrian,lost_control,crushed_into_obstacle,overturned,other,primary_accident_type,primary_accident_type_code,public_holiday,accident_type,street,locality,incident_datetime,weather,roadworks_nearby,eve_of_public_holiday
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq il-Belt Valletta,Żurrieq,2025-10-09 09:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 20.4, 'temp_min': 16.0, 'temp_max': 25.0, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq Dawret il-Gudja,Gudja,2025-06-19 18:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 24.8, 'temp_min': 21.0, 'temp_max': 29.5, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Valley Road,Qormi,2025-05-12 08:00:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 21.1, 'temp_min': 17.9, 'temp_max': 25.4, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-07-29,Tue

In [23]:
map_week = {'Monday':0,'Tuesday':1,'Wednesday':2,'Thursday':3,'Friday':4,'Saturday':5,'Sunday':6}

def to_num(day):
    return map_week.get(str(day).strip(), pd.NA)

df['day_of_week_num'] = df['day_of_week'].apply(to_num)

# helpers
def is_true(val):
    if isinstance(val,bool):
        return val
    return str(val).strip().lower() in ['true','1','yes']

is_weekend = df['day_of_week'].astype(str).str.strip().isin(['Saturday','Sunday'])

# Apply eve_ph first (Friday=4) only if not weekend
mask_eve = df['eve_of_public_holiday'].apply(is_true) & (~is_weekend)
df.loc[mask_eve, 'day_of_week_num'] = 4

# Apply ph (Sunday=6) only if not weekend
mask_ph = df['eve_of_public_holiday'].apply(is_true) & (~is_weekend)
df.loc[mask_ph, 'day_of_week_num'] = 6
df

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,affected_party,collision,running_over_pedestrian,lost_control,crushed_into_obstacle,overturned,other,primary_accident_type,primary_accident_type_code,public_holiday,accident_type,street,locality,incident_datetime,weather,roadworks_nearby,eve_of_public_holiday,day_of_week_num
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq il-Belt Valletta,Żurrieq,2025-10-09 09:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 20.4, 'temp_min': 16.0, 'temp_max': 25.0, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq Dawret il-Gudja,Gudja,2025-06-19 18:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 24.8, 'temp_min': 21.0, 'temp_max': 29.5, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Valley Road,Qormi,2025-05-12 08:00:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 21.1, 'temp_min': 17.9, 'temp_max': 25.4, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,0
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongo

In [24]:
idx_drop_rows = df.loc[(df['traffic_accident_flag']== 0) | (df['injury_severity']== 'unknown') | (df['affected_party'].isna())].index #identifying rows to be dropped
df = df.drop(idx_drop_rows)
df

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,affected_party,collision,running_over_pedestrian,lost_control,crushed_into_obstacle,overturned,other,primary_accident_type,primary_accident_type_code,public_holiday,accident_type,street,locality,incident_datetime,weather,roadworks_nearby,eve_of_public_holiday,day_of_week_num
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq il-Belt Valletta,Żurrieq,2025-10-09 09:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 20.4, 'temp_min': 16.0, 'temp_max': 25.0, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq Dawret il-Gudja,Gudja,2025-06-19 18:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 24.8, 'temp_min': 21.0, 'temp_max': 29.5, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Valley Road,Qormi,2025-05-12 08:00:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 21.1, 'temp_min': 17.9, 'temp_max': 25.4, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,0
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongo

In [25]:
blank_time = df[df['time_24h'].isna() | (df['time_24h'].astype(str).str.strip() == '')]
blank_indices = blank_time.index.tolist()
#blank_indices

df = df.drop(blank_indices)
df

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,affected_party,collision,running_over_pedestrian,lost_control,crushed_into_obstacle,overturned,other,primary_accident_type,primary_accident_type_code,public_holiday,accident_type,street,locality,incident_datetime,weather,roadworks_nearby,eve_of_public_holiday,day_of_week_num
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq il-Belt Valletta,Żurrieq,2025-10-09 09:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 20.4, 'temp_min': 16.0, 'temp_max': 25.0, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq Dawret il-Gudja,Gudja,2025-06-19 18:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 24.8, 'temp_min': 21.0, 'temp_max': 29.5, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00,"Valley Road, Qormi",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Valley Road,Qormi,2025-05-12 08:00:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 21.1, 'temp_min': 17.9, 'temp_max': 25.4, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,0
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Police investigations are ongo

In [26]:
# Define rush hour ranges
morning_start = datetime.strptime("07:30", "%H:%M").time()
morning_end = datetime.strptime("10:00", "%H:%M").time()
afternoon_start = datetime.strptime("16:00", "%H:%M").time()
afternoon_end = datetime.strptime("19:00", "%H:%M").time()

# Function to check if time falls in rush hour
def is_rush_hour(row):
    time_val = row['time_24h']
    day_num = row['day_of_week_num']
    if time_val and 0 <= day_num <= 4:  # Weekdays only
        if morning_start <= time_val <= morning_end or afternoon_start <= time_val <= afternoon_end:
            return 1
    return 0

# Apply function to create rush_hour column
df['time_24h'] = pd.to_datetime(df['time_24h'], format='%H:%M', errors='coerce').dt.time
df['rush_hour'] = df.apply(is_rush_hour, axis=1)
df

,title,date_published,date_modified,content,traffic_accident_flag,normalized_date,day_of_week,accident_day,time_24h,location,car_brand,traffic_status,injury_severity,affected_party,collision,running_over_pedestrian,lost_control,crushed_into_obstacle,overturned,other,primary_accident_type,primary_accident_type_code,public_holiday,accident_type,street,locality,incident_datetime,weather,roadworks_nearby,eve_of_public_holiday,day_of_week_num,rush_hour
0,Collision between a car and a motorbike in Żurrieq,2025-10-09,2025-10-09,"Today, at around 0930hrs, the Police were informed of a traffic accident on Triq il-Belt Valletta, Żurrieq.The Police immediately went to the scene and from a preliminary investigation it resulted that a collision had occurred between a Toyota Aqua driven by a 67-year-old woman resident of Żurrieq and a Piaggio Beverly motorbike driven by a 61-year-old man from Mqabba.A medical team was called to the scene and an ambulance took the motorbike driver to Mater Dei Hospital, where he was certified to be seriously injured.Magistrate Dr Monica Borg Galea LL.D was informed and appointed an inquiry while Police investigations are still ongoing.",1,2025-10-09,Thursday,same day,09:30:00,"Triq il-Belt Valletta, Żurrieq",Toyota,No Traffic,serious,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq il-Belt Valletta,Żurrieq,2025-10-09 09:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 20.4, 'temp_min': 16.0, 'temp_max': 25.0, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3,1
1,Car-motorcycle traffic accident,2025-06-20,2025-06-20,"Yesterday, at around 1830hrs, the Police were informed of a traffic accident in Triq Dawret il-Gudja, Gudja.Preliminary investigations showed that a collision occurred between a Honda fit that was being driven by a 64-year-old man from Santa Luċija and a Kawasaki Ninja motorcycle that was being driven by a 23-year-old man from Gudja.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-06-19,Thursday,previous day,18:30:00,"Triq Dawret il-Gudja, Gudja",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Triq Dawret il-Gudja,Gudja,2025-06-19 18:30:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 24.8, 'temp_min': 21.0, 'temp_max': 29.5, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,3,1
2,Car-motorcycle collision in Ħal Qormi,2025-05-12,2025-05-12,"Today, at around 0800hrs, the Police were informed of a traffic accident in Valley Road, Qormi.Preliminary investigations showed that a collision occurred between a Ford Transit driven by a 34-year-old man residing in Żebbuġ and a Honda motorcycle driven by a 29-year-old man who lives in Qormi.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital. He was later certified to have sustained grievous injuries.Police investigations are ongoing.",1,2025-05-12,Monday,same day,08:00:00,"Valley Road, Qormi",Honda,No Traffic,grievous,motorcyclist,0,0,0,0,0,1,other,OTH,False,Motorcycle Accident,Valley Road,Qormi,2025-05-12 08:00:00,"{'rain_today': False, 'rain_last_3h': False, 'temp_avg': 21.1, 'temp_min': 17.9, 'temp_max': 25.4, 'precip_mm_day': 0.0, 'hourly_records': 4}",False,False,0,1
3,Collision between motorcycle and car in Għaxaq,2025-07-30,2025-07-30,"Yesterday, at around 1800hrs, the Police were informed of a traffic accident in Triq Dawret Ħal Għaxaq, Għaxaq.Preliminary investigations by the Police found that there was a collision between a Volvo XC60 that was being driven by a 42-year-old man from Għaxaq and a Malaguti Madison motorcycle that was being driven by a 29-year-old Brazilian man from Zurrieq.A medical team was called on site and an ambulance took the motorcyclist to Mater Dei Hospital for treatment where he was later certified to have sustained grievous injuries.Polic